In [1]:
import numpy as np
import random
from copy import deepcopy as cp
import time

In [4]:
#reads the dataset
#operationTimes[i][j]: operation time of job i on machine j
def readSet(setName):
    f = open(setName, 'r')
    readingParams=True
    operationTimes=[]
    for line in f:
        line=line.split()
        if readingParams:
            numberOfJobs=int(line[0])
            numberOfMachines=int(line[1])
            readingParams=False
        else:
            machines=[]
            for i in range(0,numberOfMachines):
                machines.append(int(line[(2*i)+1]))
            operationTimes.append(machines)
    return operationTimes,numberOfJobs,numberOfMachines   

#generates population according to number of jobs and population size
def generatePopulation(numberOfJobs, populationSize):
    np.random.seed(4)
    population=[]
    for size in range (0,populationSize):
        population.append(np.random.permutation(numberOfJobs))
    return population

#calculates completion times of all jobs and machines
#completionTimes[i][j]: completion time of ith job in individual and machine j
#for example if individual=[2,5,6,1,0,3,4,7]
#completionTimes[1][j]: completion time of 1st job in individual (which is job 2) and machine j
#completionTimes[2][j]: completion time of 2nd job in individual (which is job 5) and machine j
def calculateCompletionTime(individual,emptyCompletionTimes,operationTimes,numberOfJobs,numberOfMachines):
    completionTimes=cp(emptyCompletionTimes)
    for k in range(0,numberOfJobs):
        job=individual[k]
        if k==0:
            for l in range (0,numberOfMachines):
                if l==0:       
                    completionTimes[k][l]=operationTimes[job][l]
                else:
                    completionTimes[k][l]=operationTimes[job][l]+completionTimes[k][l-1]
        else:
            for l in range (0,numberOfMachines):
                if l==0:
                    completionTimes[k][l]=operationTimes[job][l]+completionTimes[k-1][l]
                else:
                    completionTimes[k][l]=operationTimes[job][l]+max(completionTimes[k][l-1],completionTimes[k-1][l])
    return completionTimes
def remove_elements(l,elements):
    ### Removes elements from list
    ## elements : elements that are removed
    ## l : list that is removed from
    for e in elements:
        l.remove(e)
    return cp(l)
def cross_over(p1,p2):
    ### It generates child according to Position-Based Crossover
    ## p1 : first permutation
    ## p2 : second permutation
    ## new_child : new child of crossing-over
    l = len(p1)
    positions = np.random.choice(range(l), 3, replace=False)
    new_child = cp(p1)
    values =  [p1[i] for i in positions]
    place_positions = remove_elements(list(range(l)),positions)
    p_ = 0
    for x in p2:
        if not(x in values):
            new_child[place_positions[p_]] = x
            p_ += 1
    return new_child
def mutation(p1):
    ### p1 : permutation
    ### return : new permuatation
    ### mutation : swapping two element
    ### It makes mutations with 0.2 probability
    if 0.5 < np.random.rand():
        i,j = np.random.choice(range(len(p1)), 2, replace=False)
        temp = p1[i]
        p1[i] = p1[j]
        p1[j] = temp
        return cp(p1)
    else:
        return p1

In [6]:
#SOLUTION REPRESENTATION: PERMUTATION REPRESENTATION
#iterates over data sets
for s in range(1,4):
    #reads data set
    operationTimes,numberOfJobs,numberOfMachines=readSet('instance%s.txt' % s)
    
    #optimal values for different data sets
    optimal=0
    if s==1:
        optimal=4534
    elif s==2:
        optimal=920
    elif s==3:
        optimal=1302
    #generates random population
    populationSize=100
    population=generatePopulation(numberOfJobs, populationSize)
    completionTimes=np.zeros((numberOfJobs,numberOfMachines))
    start = time.time()
    for iteration in range(300):
        #parent selection: binary tournament method
        #select candidate parents randomly
        #2*m parents are selected
        m=10
        candidates=np.random.randint(populationSize, size=4*m)
        ## Binary tournoument manner parent selection
        parents = []
        for r in range(0,2*m):
            #randomly choose 2 parents
            can0=population[candidates[r*2]]
            can1=population[candidates[(r*2)+1]]
            #calculate completion times and makespans
            cT0=calculateCompletionTime(can0,completionTimes,operationTimes,numberOfJobs,numberOfMachines)
            makespan0=cT0[numberOfJobs-1][numberOfMachines-1]
            cT1=calculateCompletionTime(can1,completionTimes,operationTimes,numberOfJobs,numberOfMachines)
            makespan1=cT1[numberOfJobs-1][numberOfMachines-1] 
            if makespan0<makespan1:
                parents.append(cp(can0))
            else:
                parents.append(cp(can1))
        ## Children list
        children = []
        ### Children are generated with mutation and cross-over
        for i in range(0,2*m,2):
            p1 = parents[i]
            p2 = parents[i+1]
            children.append(mutation(cross_over(p1,p2)))
            children.append(mutation(cross_over(p2,p1)))
        ### Find worst solutions
        make_span_list = []
        for p in population:
            ct = calculateCompletionTime(p,completionTimes,operationTimes,numberOfJobs,numberOfMachines)
            make_span_list.append(cp(ct[numberOfJobs-1][numberOfMachines-1]))
        worst = sorted(range(len(make_span_list)), key=lambda k: make_span_list[k])[20:40]
        ### Add worst ones to childeren list
        for i in worst:
                children.append(population[i])
        ##########
        ##########
        #### Worst are eliminated from population
        new_population = []
        for i,x in enumerate(population):
                if not(i in worst):
                    new_population.append(x)
        ##########
        ##########
        ### Children and worst ones are sorted
        ### Best 20 added to population
        make_span_list = []
        for p in children:
            ct = calculateCompletionTime(p,completionTimes,operationTimes,numberOfJobs,numberOfMachines)
            make_span_list.append(ct[numberOfJobs-1][numberOfMachines-1])
        best = sorted(range(len(make_span_list)), key=lambda k: make_span_list[k])[0:20]
        for i in best:
            new_population.append(children[i])
        ########
        ### New population for next iteration
        population = cp(new_population)
    ####
    # Find statistics
    cpu_usage = (time.time()-start)
    make_span_list = []
    for p in population:
        ct = calculateCompletionTime(p,completionTimes,operationTimes,numberOfJobs,numberOfMachines)
        make_span_list.append(cp(ct[numberOfJobs-1][numberOfMachines-1]))
    best = min(make_span_list)
    best_index = make_span_list.index(best)
    avg = sum(make_span_list)/len(make_span_list)
    print('-----------------------------------------')
    print('-----------------------------------------')
    print('-----------------------------------------')
    print('Best Sequence:',population[best_index])
    print('Best:', best)
    print('Best Dev:',(best - optimal) / optimal)
    print('Avg:', avg)
    print('Avg Dev:',(avg - optimal) / optimal)
    print('CPU Usage:',cpu_usage)

-----------------------------------------
-----------------------------------------
-----------------------------------------
Best Sequence: [7 4 2 6 1 5 0 3]
Best: 4534.0
Best Dev: 0.0
Avg: 5284.23
Avg Dev: 0.16546757829730913
CPU Usage: 2.437532424926758
-----------------------------------------
-----------------------------------------
-----------------------------------------
Best Sequence: [ 2  5  6  4 10  0 11  8  9 12  7  1  3]
Best: 937.0
Best Dev: 0.01847826086956522
Avg: 1063.42
Avg Dev: 0.15589130434782617
CPU Usage: 4.071516513824463
-----------------------------------------
-----------------------------------------
-----------------------------------------
Best Sequence: [ 3 15  8 17 11  6 19 18 12 13  7  4  2 16  9  0 14  5 10  1]
Best: 1302.0
Best Dev: 0.0
Avg: 1484.31
Avg Dev: 0.14002304147465433
CPU Usage: 5.900924921035767
